In [ ]:
!pip install emoji

In [ ]:
!pip install sklearn-deap

In [ ]:
!pip install underthesea

In [ ]:
import pandas as pd
import numpy as np
import underthesea
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import sklearn
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from tqdm import tqdm_notebook
import torch
import transformers
import joblib
import emoji

In [ ]:
data = pd.read_excel('../input/nlpdataver2/crawl-nlp-tiki-ver2.xlsx',header=None)
print(len(data))

In [ ]:
data

In [ ]:
def clean_str(string):
    string = re.sub(r"[^aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9]", " ", string)
    string = give_emoji_free_text(string)
    return string.strip()

def give_emoji_free_text(text):
    for i in text:
        if i in emoji.UNICODE_EMOJI:
            text = text.replace(i,'')
    return text

def remove_blank(s):
    re = ''
    for i in range(0,len(s)):
        if s[i] == ' ':
            if s[i-1] != ' ':
                re = re + s[i]
        else:
            re = re + s[i]
    return re

def text_lowercase(string):
    return string.lower()

def tokenize(strings):    
    return underthesea.word_tokenize(strings, format="text")

def remove_stopwords(strings):
    strings = strings.split()
    f = open('../input/vietnamesestopword/vietnamese-stopwords.txt', 'r',encoding="utf-8")
    stopwords = f.readlines()
    stop_words = [s.replace("\n", '') for s in stopwords]
    doc_words = []
    
    for word in strings:
        if word not in stop_words:
            doc_words.append(word)
    doc_str = ' '.join(doc_words).strip()
    return doc_str

def text_preprocessing(strings):
    temp = clean_str(strings)
    temp = text_lowercase(temp)
    temp = tokenize(temp)
#     temp = remove_stopwords(temp)
    return temp

def text_preprocessing2(strings):
    temp = clean_str(strings)
    temp = text_lowercase(temp)
    temp = remove_stopwords(temp)
    return temp

def removeEmptySentens(data):
    data[0].replace('', np.nan, inplace=True)
    data.dropna(subset=[0], inplace=True)


In [ ]:
temp_id = []
data_processbert = []
data_processtfidf = []
count = 0
for i in data[0]:
    data_processbert.append(text_preprocessing(i))
    data_processtfidf.append(text_preprocessing2(i))
    if data_processbert[count] == '' or data_processtfidf[count] == '':
        print(count)
        temp_id.append(count)
    count = count + 1
data[0] = pd.DataFrame(data_processbert)

In [ ]:
len(data)

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

In [ ]:
count = 0
data = data.drop(temp_id)
for i in temp_id:
    data_processtfidf.pop(i-count)
    count = count + 1

In [ ]:
len(data_processtfidf)

In [ ]:
len(data)

In [ ]:
removeEmptySentens(data)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_processtfidf).toarray()

In [ ]:
X.shape

In [ ]:
phobert = transformers.AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = transformers.AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
#encode lines
tokenized = data[0].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))
print('encode',tokenized[1])
# decode
print('decode',tokenizer.decode(tokenized[1]))

In [ ]:
labels = data[1]
print('labels shape:', labels.shape)

# get lenght max of tokenized
max_len = 254
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)
print('max len:', max_len)

# if lenght of tokenized not equal max_len , so padding value 0
padded = []
for i in tokenized.values:
    if len(i) > max_len:
        padded.append(i[0:max_len])
    else:
        padded.append(i + [0]*(max_len-len(i)))
# padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

padded = np.array(padded)
print('padded:', padded[1])
print('len padded:', np.array(padded).shape)

#get attention mask ( 0: not has word, 1: has word)
attention_mask = np.where(padded ==0, 0,1)

# Convert input to tensor
padded = torch.tensor(padded).cuda()
attention_mask = torch.tensor(attention_mask).cuda()

In [ ]:
labels = data[1].tolist()

In [ ]:
labels_tensor = torch.tensor(labels).cuda()

In [ ]:
len(padded[0])

In [ ]:
attention_mask.shape

In [ ]:
data = TensorDataset(padded, attention_mask, labels_tensor)
sampler = SequentialSampler(data)
dataloader = DataLoader(data, sampler=sampler, batch_size=32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
phobert.cuda()

In [ ]:

# Train model
phobert.eval()
all_features = []
with torch.no_grad():
    for index,batch in tqdm_notebook(enumerate(dataloader)):
        batch = tuple(t.to(device) for t in batch)
        padded,attention_mask,_ = batch
        last_hidden_states = phobert(padded, attention_mask =attention_mask)
        all_features.extend(last_hidden_states[1].tolist().copy())
        print(index)
        print(last_hidden_states[1].shape)

In [ ]:
np.asarray(all_features).shape

In [ ]:
np.asarray(labels).shape

In [ ]:
print(type(X))
print(X.shape)

In [ ]:
np.asarray(all_features).shape

In [ ]:
features = np.concatenate((all_features,X), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels,test_size=0.1)

cl = LogisticRegression()
cl.fit(X_train, y_train)

# Save model
joblib.dump(cl, 'save_model_nlp_logistic.pkl')
print("Classification Report: \n {}\n".format(classification_report(y_test, cl.predict(X_test))))

In [ ]:
# rf_params = {
#     'C': np.random.uniform(0,50,1000),
#     "kernel":['linear','poly','rbf','sigmoid']
# }
# clf = sklearn.svm.SVC(gamma='scale')
# ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
#                                    params=rf_params,
#                                    scoring="accuracy",
#                                    cv=3,
#                                    verbose=1,
#                                    population_size=10,
#                                    gene_mutation_prob=0.10,
#                                    gene_crossover_prob=0.5,
#                                    tournament_size=3,
#                                    generations_number=5,
#                                    n_jobs=1)
# ga1.fit(X_train, y_train)
# print(ga1.best_params_)
# print("Accuracy:"+ str(ga1.best_score_))

In [ ]:
svm = sklearn.svm.SVC(C = 5.80912162343139,kernel = 'rbf')
svm.fit(X_train, y_train)

joblib.dump(cl, 'save_model_nlp_svm.pkl')
print("Classification Report: \n {}\n".format(classification_report(y_test, svm.predict(X_test))))

In [ ]:
# # Define the hyperparameter configuration space
# rf_params = {
#     'n_estimators': [10, 20, 30],
#     #'max_features': ['sqrt',0.5],
#     'max_depth': [15,20,30,50],
#     #'min_samples_leaf': [1,2,4,8],
#     #"bootstrap":[True,False],
#     "criterion":['gini','entropy']
# }
# clf = RandomForestClassifier(random_state=0)
# grid = GridSearchCV(clf, rf_params, cv=3, scoring='accuracy')
# grid.fit(X_test, y_test)
# print(grid.best_params_)
# print("Accuracy:"+ str(grid.best_score_))

In [ ]:
rf= RandomForestClassifier(n_estimators = 20, criterion='entropy',max_depth=15)
rf.fit(X_train, y_train)

joblib.dump(rf, 'save_model_nlp_rf.pkl')
print("Classification Report: \n {}\n".format(classification_report(y_test, rf.predict(X_test))))